In [2]:
import gspread
from google.oauth2.service_account import Credentials
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import time
import openai

In [ ]:
driver_path = 'chromedriver.exe'
openai.api_key = 'example-key'
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
sheet_path = "sheets-5e3cf7f4981a.json"
sheet_url = "example-url"

Retrieving URL:

In [11]:
gc = gspread.service_account(filename=sheet_path)
sh = gc.open_by_url(sheet_url)
worksheet = sh.get_worksheet(0)
first_url = worksheet.col_values(2)[3]

print(first_url)

https://www.upwork.com/jobs/~010b21a1e25f531396


Using 'Tickbox' Logic to only retrieve New jobs:

In [12]:
gc = gspread.service_account(filename=sheet_path)
sh = gc.open_by_url(sheet_url)
worksheet = sh.get_worksheet(0)

urls_column = worksheet.col_values(2)[2:]  # URLs start from B3, skipping header
done_column = worksheet.col_values(16)[2:]  # DONE column starts from P3, column P is the 16th column

first_unchecked_url = None
for url, done in zip(urls_column, done_column):
    if done.upper() == "FALSE":  # Checking if the 'DONE' column is unchecked
        first_unchecked_url = url
        break

print(first_unchecked_url)

None


Adding scraping and update sheet functionality- Job Title:

In [13]:
gc = gspread.service_account(filename=sheet_path)
sh = gc.open_by_url(sheet_url)
worksheet = sh.get_worksheet(0)

urls_column = worksheet.col_values(2)[2:]  # URLs start from B3
done_column = worksheet.col_values(16)[2:]  # DONE column starts from P3
titles_column = worksheet.col_values(6)[2:]  # Titles column starts from F3

first_unchecked_row = None
first_unchecked_url = None

# Loop through the URLs and DONE column to find the first unchecked row
for i, (url, done) in enumerate(zip(urls_column, done_column)):
    if done.upper() == "FALSE":
        first_unchecked_url = url
        first_unchecked_row = i + 3  # Adjust to match row number in sheet (starting from row 3)
        break

if first_unchecked_url:
    # Initialize Selenium WebDriver
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    
    # Navigate to the URL
    driver.get(first_unchecked_url)
    time.sleep(3)  # Wait for page load
    
    # Scrape the title using the class name
    try:
        job_title_element = driver.find_element(By.CSS_SELECTOR, "h4.m-0")
        job_title = job_title_element.text
    except:
        job_title = "Title not found"
    
    driver.quit()

    # Update the 'Title' column (column F) with the scraped job title
    worksheet.update_cell(first_unchecked_row, 6, job_title)  # F column is 6th column
else:
    print("No unchecked URL found.")


No unchecked URL found.


Title, Proj Type, and price:

In [ ]:
from selenium.webdriver.common.by import By

gc = gspread.service_account(filename=sheet_path)
sh = gc.open_by_url(sheet_url)
worksheet = sh.get_worksheet(0)

urls_column = worksheet.col_values(2)[2:200]  # URLs start from B3, go till B200
done_column = worksheet.col_values(17)[2:200]  # DONE column now in Q3, column Q is the 17th column, go till Q200

first_unchecked_row = None
first_unchecked_url = None

# Loop through the DONE column up to row 200 to find the first unchecked row
for i, (url, done) in enumerate(zip(urls_column, done_column)):
    if done.upper() == "FALSE":
        first_unchecked_url = url
        first_unchecked_row = i + 3  # Adjust to match row number in sheet (starting from row 3)
        break

if first_unchecked_url:
    # Initialize Selenium WebDriver
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    
    # Navigate to the URL
    driver.get(first_unchecked_url)
    time.sleep(3)  # Wait for page load
    
    # Scrape the job title using the class name
    try:
        job_title_element = driver.find_element(By.CSS_SELECTOR, "h4.m-0")
        job_title = job_title_element.text
    except:
        job_title = "Title not found"
    
    # Update the 'Title' column (column F) with the scraped job title
    worksheet.update_cell(first_unchecked_row, 6, job_title)  # F column is 6th column

    # Try to find the element with "data-v-8d6ae40e" which indicates budget info
    try:
        budget_element = driver.find_element(By.CSS_SELECTOR, "[data-v-8d6ae40e]")
        
        # Check if it's hourly or fixed price based on certain keywords
        job_type = "Hourly" if "Hourly" in budget_element.text else "Fixed Price"
        
        # Update the 'Job Type' column (column L)
        worksheet.update_cell(first_unchecked_row, 12, job_type)  # L column is 12th column
        
        # If it's a fixed price job, extract and store the price in column I
        if job_type == "Fixed Price":
            try:
                price_element = budget_element.find_element(By.TAG_NAME, "strong")
                price = price_element.text
                worksheet.update_cell(first_unchecked_row, 9, price)  # I column is 9th column
            except:
                pass  # If no price is found, don't update the price column
    except:
        pass  # If no budget info is found, do nothing
    
    driver.quit()
else:
    print("No unchecked URL found in the range Q3 to Q200.")


Adding Skills, and activity section details:

In [65]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

gc = gspread.service_account(filename=sheet_path)
sh = gc.open_by_url(sheet_url)
worksheet = sh.get_worksheet(0)

urls_column = worksheet.col_values(2)[2:200]  # URLs start from B3
done_column = worksheet.col_values(22)[2:200]  # DONE column starts from Q3 (22nd column)

first_unchecked_row = None
first_unchecked_url = None

# Find the first unchecked URL
for i, (url, done) in enumerate(zip(urls_column, done_column)):
    if done.upper() == "FALSE":
        first_unchecked_url = url
        first_unchecked_row = i + 3
        break

if first_unchecked_url:
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    
    driver.get(first_unchecked_url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Scraping Project Type, Price, and Skills
    try:
        # Scrape the job title
        job_title_element = driver.find_element(By.CSS_SELECTOR, "h4.m-0")
        job_title = job_title_element.text
    except:
        job_title = "Title not found"
    
    worksheet.update_cell(first_unchecked_row, 6, job_title)  # Update job title in F

    try:
        # Check for Hourly jobs first by detecting 'data-cy="clock-hourly"' element
        try:
            hourly_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='clock-hourly']")
            job_type = "Hourly"
            worksheet.update_cell(first_unchecked_row, 11, job_type)  # Update Project Type in K
            
            # Extract both minimum and maximum hourly rate
            try:
                rate_elements = driver.find_elements(By.CSS_SELECTOR, "div[data-test='BudgetAmount'] strong")
                
                if len(rate_elements) == 2:
                    min_rate = rate_elements[0].text.strip()  # Extract minimum rate
                    max_rate = rate_elements[1].text.strip()  # Extract maximum rate
                    hourly_rate = f"{min_rate} - {max_rate}"
                    worksheet.update_cell(first_unchecked_row, 8, hourly_rate)  # Update Price in H
            except Exception as e:
                print(f"Error extracting hourly rate: {e}")  # No hourly rate found
            
        except:
            # If Hourly job is not detected, check for Fixed Price jobs (same logic as before)
            try:
                budget_element = driver.find_element(By.CSS_SELECTOR, "[data-v-8d6ae40e]")
                job_type = "Fixed Price"
                worksheet.update_cell(first_unchecked_row, 11, job_type)  # Update Project Type in K

                # If it's Fixed Price, extract the price
                price_element = budget_element.find_element(By.TAG_NAME, "strong")
                price = price_element.text
                worksheet.update_cell(first_unchecked_row, 8, price)  # Update Price in H

            except:
                pass  # No Fixed Price or Hourly found
    except Exception as e:
        print(f"Error in scraping job type and price: {e}")
        pass

    try:
        # Scrape the skills section
        skills_sections = driver.find_elements(By.CSS_SELECTOR, "div.skills-list.mt-3")
        all_skills = []
        for section in skills_sections:
            skill_elements = section.find_elements(By.CSS_SELECTOR, "span.air3-badge.air3-badge-highlight.badge.disabled")
            section_skills = [skill.text for skill in skill_elements]
            all_skills.extend(section_skills)
        
        # Ensure unique skills and join them in a comma-separated string
        unique_skills = list(set(all_skills))
        skills_text = ", ".join(unique_skills)
        worksheet.update_cell(first_unchecked_row, 12, skills_text)  # Update Skills in L
    except:
        pass

    # Scraping Activity Section
    try:
        # Find the activity section using the specific class
        activity_section = driver.find_element(By.CSS_SELECTOR, "ul.client-activity-items.list-unstyled.visitor")

        # Scrape Proposals
        try:
            proposals = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Proposals')]/following-sibling::span[@class='value']").text
            worksheet.update_cell(first_unchecked_row, 13, proposals)  # Update Proposals in M
        except:
            proposals = "Not found"

        # Scrape Last viewed by client
        try:
            last_viewed = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Last viewed by client')]/following-sibling::span[@class='value']").text
            worksheet.update_cell(first_unchecked_row, 14, last_viewed)  # Update Last viewed in N
        except:
            last_viewed = "Not found"

        # Scrape Interviewing
        try:
            interviewing = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Interviewing')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, 15, interviewing)  # Update Interviewing in O
        except:
            interviewing = "Not found"

        # Scrape Invites sent
        try:
            invites_sent = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Invites sent')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, 16, invites_sent)  # Update Invites sent in P
        except:
            invites_sent = "Not found"

        # Scrape Unanswered invites
        try:
            unanswered_invites = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Unanswered invites')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, 17, unanswered_invites)  # Update Unanswered invites in Q
        except:
            unanswered_invites = "Not found"

    except:
        print("Could not find the 'Activity on this job' section.")

    driver.quit()
else:
    print("No unchecked URL found in the range Q3 to Q200.")


Adding bs4 code to deal with certain issues:

In [44]:
soup

<html class="cq-nav-desktop" data-gl-app="" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en-us%22%7D,%22theme%22:%7B%22ssr%22:%22air-3-0%22%7D%7D" lang="en-us" theme="air-3-0"><head><script async="" src="https://www.googletagmanager.com/gtag/js?id=AW-972598239&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="//static.ads-twitter.com/uwt.js"></script><script async="" src="//bat.bing.com/bat.js"></script><script async="" id="pdst-capture" src="https://cdn.pdst.fm/ping.min.js"></script><script async="" src="https://d.impactradius-event.com/A2798569-68a2-43d5-98ff-96c66d56992b1.js"></script><script async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-972598239&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-11023177336&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://a.quora.com/qevents.js" type="text/javascript"></scri

In [61]:
prices = soup.find_all('strong', attrs={'data-v-8d6ae40e': True})

for price in prices:
    print(price.get_text())


      $20.00
    

      $40.00
    


In [58]:
price

[<strong class="nav-item-label"><span></span><!-- --></strong>,
 <strong class="nav-item-label vs-color-text">Find talent</strong>,
 <strong class="nav-item-label vs-color-text">Post a job and hire a pro</strong>,
 <strong class="nav-item-label vs-color-text">Browse and buy projects</strong>,
 <strong class="nav-item-label vs-color-text">Get advice from an industry expert</strong>,
 <strong class="vs-color-text">Post a job and hire a pro</strong>,
 <strong class="d-block ml-6x mt-6x vs-text-heading vs-color-text" data-cy="item-header"><span>Talent Marketplace</span><sup class="vertical-align-top vs-text-sup">TM</sup></strong>,
 <strong class="vs-color-text">Browse and buy projects</strong>,
 <strong class="d-block ml-6x mt-6x vs-text-heading vs-color-text" data-cy="item-header"><span>Project Catalog</span><sup class="vertical-align-top vs-text-sup">TM</sup></strong>,
 <strong class="vs-color-text">Get advice from an industry expert</strong>,
 <strong class="d-block ml-6x mt-6x vs-text-

Using Column names instead of Lettering:

In [70]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

gc = gspread.service_account(filename=sheet_path)
sh = gc.open_by_url(sheet_url)
worksheet = sh.get_worksheet(0)

# Read the column names from Row 2
column_headers = worksheet.row_values(2)

# Create a dictionary mapping column names to column indexes
columns = {header: idx + 1 for idx, header in enumerate(column_headers)}

# Example column names based on your provided structure
# Columns are now dynamically found by name
name_col = columns.get("Name")
url_col = columns.get("URL")
title_col = columns.get("Title")
price_col = columns.get("Price")
project_type_col = columns.get("Project Type")
skills_col = columns.get("Skills")
proposals_col = columns.get("Proposals")
last_viewed_col = columns.get("Last viewed")
interviewing_col = columns.get("Interviewing")
invites_sent_col = columns.get("Invites sent")
unanswered_invites_col = columns.get("Unanswered invites")
done_col = columns.get("Done")

# Get the values from the "URL" column starting from Row 3
urls_column = worksheet.col_values(url_col)[2:200]  # URLs start from Row 3
done_column = worksheet.col_values(done_col)[2:200]  # DONE column starts from Row 3

first_unchecked_row = None
first_unchecked_url = None

# Find the first unchecked URL
for i, (url, done) in enumerate(zip(urls_column, done_column)):
    if done.upper() == "FALSE":
        first_unchecked_url = url
        first_unchecked_row = i + 3
        break

if first_unchecked_url:
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    
    driver.get(first_unchecked_url)
    time.sleep(3)

    # Scraping Project Type, Price, and Skills
    try:
        # Scrape the job title
        job_title_element = driver.find_element(By.CSS_SELECTOR, "h4.m-0")
        job_title = job_title_element.text
        worksheet.update_cell(first_unchecked_row, title_col, job_title)  # Update job title in the "Title" column
    except:
        job_title = "Title not found"

    # Use BeautifulSoup to scrape the hourly rate if the job is hourly
    try:
        # Check for Hourly jobs by detecting 'data-cy="clock-hourly"' element
        hourly_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='clock-hourly']")
        job_type = "Hourly"
        worksheet.update_cell(first_unchecked_row, project_type_col, job_type)  # Update Project Type in the corresponding column

        # Use BeautifulSoup to extract the hourly rate
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        prices = soup.find_all('strong', attrs={'data-v-8d6ae40e': True})
        
        if len(prices) == 2:
            min_price = prices[0].get_text().strip()
            max_price = prices[1].get_text().strip()
            hourly_rate = f"{min_price} - {max_price}"
            worksheet.update_cell(first_unchecked_row, price_col, hourly_rate)  # Update Price in the "Price" column
        else:
            worksheet.update_cell(first_unchecked_row, price_col, "No hourly rate provided")  # No rate available
    except:
        # If it's not an hourly job, fallback to fixed price extraction
        try:
            budget_element = driver.find_element(By.CSS_SELECTOR, "[data-v-8d6ae40e]")
            job_type = "Fixed Price"
            worksheet.update_cell(first_unchecked_row, project_type_col, job_type)  # Update Project Type in the corresponding column

            # Extract the fixed price
            price_element = budget_element.find_element(By.TAG_NAME, "strong")
            price = price_element.text
            worksheet.update_cell(first_unchecked_row, price_col, price)  # Update Price in the "Price" column
        except:
            pass

    # Scraping Skills Section (using Selenium)
    try:
        skills_sections = driver.find_elements(By.CSS_SELECTOR, "div.skills-list.mt-3")
        all_skills = []
        for section in skills_sections:
            skill_elements = section.find_elements(By.CSS_SELECTOR, "span.air3-badge.air3-badge-highlight.badge.disabled")
            section_skills = [skill.text for skill in skill_elements]
            all_skills.extend(section_skills)
        
        # Ensure unique skills and join them in a comma-separated string
        unique_skills = list(set(all_skills))
        skills_text = ", ".join(unique_skills)
        worksheet.update_cell(first_unchecked_row, skills_col, skills_text)  # Update Skills in the "Skills" column
    except:
        pass

    # Scraping Activity Section (via Selenium, no changes here)
    try:
        # Find the activity section using the specific class
        activity_section = driver.find_element(By.CSS_SELECTOR, "ul.client-activity-items.list-unstyled.visitor")

        # Scrape Proposals
        try:
            proposals = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Proposals')]/following-sibling::span[@class='value']").text
            worksheet.update_cell(first_unchecked_row, proposals_col, proposals)  # Update Proposals in the "Proposals" column
        except:
            proposals = "Not found"

        # Scrape Last viewed by client
        try:
            last_viewed = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Last viewed by client')]/following-sibling::span[@class='value']").text
            worksheet.update_cell(first_unchecked_row, last_viewed_col, last_viewed)  # Update Last viewed in the "Last viewed" column
        except:
            last_viewed = "Not found"

        # Scrape Interviewing
        try:
            interviewing = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Interviewing')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, interviewing_col, interviewing)  # Update Interviewing in the "Interviewing" column
        except:
            interviewing = "Not found"

        # Scrape Invites sent
        try:
            invites_sent = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Invites sent')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, invites_sent_col, invites_sent)  # Update Invites sent in the "Invites sent" column
        except:
            invites_sent = "Not found"

        # Scrape Unanswered invites
        try:
            unanswered_invites = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Unanswered invites')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, unanswered_invites_col, unanswered_invites)  # Update Unanswered invites in the "Unanswered invites" column
        except:
            unanswered_invites = "Not found"

    except:
        print("Could not find the 'Activity on this job' section.")

    driver.quit()
else:
    print("No unchecked URL found in the range Q3 to Q200.")


Adding Experience Level, Duration:

In [74]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

gc = gspread.service_account(filename=sheet_path)
sh = gc.open_by_url(sheet_url)
worksheet = sh.get_worksheet(0)

# Read the column names from Row 2
column_headers = worksheet.row_values(2)

# Create a dictionary mapping column names to column indexes
columns = {header: idx + 1 for idx, header in enumerate(column_headers)}

# Example column names based on your provided structure
name_col = columns.get("Name")
url_col = columns.get("URL")
title_col = columns.get("Title")
payment_type_col = columns.get("Payment Type")  # Updated to Payment Type
price_col = columns.get("Price")
project_type_col = columns.get("Project Type")  # Updated for actual job type (e.g., Ongoing project)
skills_col = columns.get("Skills")
proposals_col = columns.get("Proposals")
last_viewed_col = columns.get("Last viewed")
interviewing_col = columns.get("Interviewing")
invites_sent_col = columns.get("Invites sent")
unanswered_invites_col = columns.get("Unanswered invites")
done_col = columns.get("Done")
experience_level_col = columns.get("Experience Level")
duration_col = columns.get("Duration")

# Get the values from the "URL" column starting from Row 3
urls_column = worksheet.col_values(url_col)[2:200]  # URLs start from Row 3
done_column = worksheet.col_values(done_col)[2:200]  # DONE column starts from Row 3

first_unchecked_row = None
first_unchecked_url = None

# Find the first unchecked URL
for i, (url, done) in enumerate(zip(urls_column, done_column)):
    if done.upper() == "FALSE":
        first_unchecked_url = url
        first_unchecked_row = i + 3
        break

if first_unchecked_url:
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    
    driver.get(first_unchecked_url)
    time.sleep(3)

    # Scraping Project Type, Payment Type, Price, and Skills
    try:
        # Scrape the job title
        job_title_element = driver.find_element(By.CSS_SELECTOR, "h4.m-0")
        job_title = job_title_element.text
        worksheet.update_cell(first_unchecked_row, title_col, job_title)  # Update job title in the "Title" column
    except:
        job_title = "Title not found"

    # Use BeautifulSoup to scrape the hourly rate if the job is hourly
    try:
        # Check for Hourly jobs by detecting 'data-cy="clock-hourly"' element
        hourly_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='clock-hourly']")
        payment_type = "Hourly"
        worksheet.update_cell(first_unchecked_row, payment_type_col, payment_type)  # Update Payment Type in the corresponding column

        # Use BeautifulSoup to extract the hourly rate
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        prices = soup.find_all('strong', attrs={'data-v-8d6ae40e': True})
        
        if len(prices) == 2:
            min_price = prices[0].get_text().strip()
            max_price = prices[1].get_text().strip()
            hourly_rate = f"{min_price} - {max_price}"
            worksheet.update_cell(first_unchecked_row, price_col, hourly_rate)  # Update Price in the "Price" column
        else:
            worksheet.update_cell(first_unchecked_row, price_col, "No hourly rate provided")  # No rate available
    except:
        # If it's not an hourly job, fallback to fixed price extraction
        try:
            budget_element = driver.find_element(By.CSS_SELECTOR, "[data-v-8d6ae40e]")
            payment_type = "Fixed Price"
            worksheet.update_cell(first_unchecked_row, payment_type_col, payment_type)  # Update Payment Type in the corresponding column

            # Extract the fixed price
            price_element = budget_element.find_element(By.TAG_NAME, "strong")
            price = price_element.text
            worksheet.update_cell(first_unchecked_row, price_col, price)  # Update Price in the "Price" column
        except:
            pass

    # Scraping Skills Section (using Selenium)
    try:
        skills_sections = driver.find_elements(By.CSS_SELECTOR, "div.skills-list.mt-3")
        all_skills = []
        for section in skills_sections:
            skill_elements = section.find_elements(By.CSS_SELECTOR, "span.air3-badge.air3-badge-highlight.badge.disabled")
            section_skills = [skill.text for skill in skill_elements]
            all_skills.extend(section_skills)
        
        # Ensure unique skills and join them in a comma-separated string
        unique_skills = list(set(all_skills))
        skills_text = ", ".join(unique_skills)
        worksheet.update_cell(first_unchecked_row, skills_col, skills_text)  # Update Skills in the "Skills" column
    except:
        pass

    # Scraping Duration
    try:
        duration_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='duration2'] + strong")
        duration = duration_element.text
        worksheet.update_cell(first_unchecked_row, duration_col, duration)  # Update Duration in the "Duration" column
    except:
        duration = "Not found"

    # Scraping Experience Level
    try:
        experience_level_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='expertise'] + strong")
        experience_level = experience_level_element.text
        worksheet.update_cell(first_unchecked_row, experience_level_col, experience_level)  # Update Experience Level in the "Experience Level" column
    except:
        experience_level = "Not found"

    # Scraping Project Type (e.g., Ongoing project, etc.)
    try:
        project_type_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='briefcase-outlined'] + strong")
        project_type = project_type_element.text
        worksheet.update_cell(first_unchecked_row, project_type_col, project_type)  # Update Project Type in the "Project Type" column
    except:
        project_type = "Not found"

    # Scraping Activity Section (via Selenium, no changes here)
    try:
        # Find the activity section using the specific class
        activity_section = driver.find_element(By.CSS_SELECTOR, "ul.client-activity-items.list-unstyled.visitor")

        # Scrape Proposals
        try:
            proposals = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Proposals')]/following-sibling::span[@class='value']").text
            worksheet.update_cell(first_unchecked_row, proposals_col, proposals)  # Update Proposals in the "Proposals" column
        except:
            proposals = "Not found"

        # Scrape Last viewed by client
        try:
            last_viewed = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Last viewed by client')]/following-sibling::span[@class='value']").text
            worksheet.update_cell(first_unchecked_row, last_viewed_col, last_viewed)  # Update Last viewed in the "Last viewed" column
        except:
            last_viewed = "Not found"

        # Scrape Interviewing
        try:
            interviewing = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Interviewing')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, interviewing_col, interviewing)  # Update Interviewing in the "Interviewing" column
        except:
            interviewing = "Not found"

        # Scrape Invites sent
        try:
            invites_sent = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Invites sent')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, invites_sent_col, invites_sent)  # Update Invites sent in the "Invites sent" column
        except:
            invites_sent = "Not found"

        # Scrape Unanswered invites
        try:
            unanswered_invites = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Unanswered invites')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, unanswered_invites_col, unanswered_invites)  # Update Unanswered invites in the "Unanswered invites" column
        except:
            unanswered_invites = "Not found"

    except:
        print("Could not find the 'Activity on this job' section.")

    driver.quit()
else:
    print("No unchecked URL found in the range Q3 to Q200.")


Adding 'About the client' details:

In [ ]:
import gspread
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

driver_path = 'chromedriver.exe'
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
sheet_path = "sheets-5e3cf7f4981a.json"
sheet_url = "example-url"

# Gspread and Google Sheets authentication
gc = gspread.service_account(filename=sheet_path)
sh = gc.open_by_url(sheet_url)
worksheet = sh.get_worksheet(0)

columns = {header: idx + 1 for idx, header in enumerate(column_headers)}

In [12]:
name_col = columns.get("Name")
url_col = columns.get("URL")
title_col = columns.get("Title")
payment_type_col = columns.get("Payment Type")
price_col = columns.get("Price")
project_type_col = columns.get("Project Type")
skills_col = columns.get("Skills")
proposals_col = columns.get("Proposals")
last_viewed_col = columns.get("Last viewed")
interviewing_col = columns.get("Interviewing")
invites_sent_col = columns.get("Invites sent")
unanswered_invites_col = columns.get("Unanswered invites")
done_col = columns.get("Done")
experience_level_col = columns.get("Experience Level")
duration_col = columns.get("Duration")
member_since_col = columns.get("Member Since")
location_col = columns.get("Location")
total_spent_col = columns.get("Total Spent")
hires_col = columns.get("Hires")
client_type_col = columns.get("Client Type")
description_col = columns.get("Description")

first_unchecked_row = None
first_unchecked_url = None

In [ ]:
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service


# Authenticate and access the Google Sheet
gc = gspread.service_account(filename=sheet_path)
sh = gc.open_by_url(sheet_url)
worksheet = sh.get_worksheet(0)

# Retrieve the column headers and map them to column indices
column_headers = worksheet.row_values(2)
columns = {header: idx + 1 for idx, header in enumerate(column_headers)}

# Assign column numbers to variables for easier access
name_col = columns.get("Name")
url_col = columns.get("URL")
title_col = columns.get("Title")
payment_type_col = columns.get("Payment Type")
price_col = columns.get("Price")
project_type_col = columns.get("Project Type")
skills_col = columns.get("Skills")
proposals_col = columns.get("Proposals")
last_viewed_col = columns.get("Last viewed")
interviewing_col = columns.get("Interviewing")
invites_sent_col = columns.get("Invites sent")
unanswered_invites_col = columns.get("Unanswered invites")
done_col = columns.get("Done")
experience_level_col = columns.get("Experience Level")
duration_col = columns.get("Duration")
member_since_col = columns.get("Member Since")
location_col = columns.get("Location")
total_spent_col = columns.get("Total Spent")
hires_col = columns.get("Hires")
client_type_col = columns.get("Client Type")
description_col = columns.get("Description")

# Get the URLs and Done status from the relevant columns
urls_column = worksheet.col_values(url_col)[2:200]
done_column = worksheet.col_values(done_col)[2:200]

# Loop through the URLs starting from the first unchecked row
for i, (url, done) in enumerate(zip(urls_column, done_column)):
    # Stop when a "TRUE" box is found (i.e., ticked box in 'Done' column)
    if done.upper() == "TRUE":
        print(f"Stopping at row {i + 3}, as it's already marked 'Done'")
        break
    
    first_unchecked_row = i + 3  # Since data starts from row 3
    
    # If URL is not empty, proceed to scrape
    if url:
        service = Service(driver_path)
        driver = webdriver.Chrome(service=service)
        
        driver.get(url)
        time.sleep(3)

        try:
            # Scrape the job title
            job_title_element = driver.find_element(By.CSS_SELECTOR, "h4.m-0")
            job_title = job_title_element.text
            worksheet.update_cell(first_unchecked_row, title_col, job_title)
        except:
            pass

        try:
            # Scrape job description
            description_element = driver.find_element(By.CSS_SELECTOR, "div.break.mt-2 p.text-body-sm")
            job_description = description_element.text
            worksheet.update_cell(first_unchecked_row, description_col, job_description)
        except:
            pass

        try:
            # Check if the job is hourly or fixed price
            hourly_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='clock-hourly']")
            payment_type = "Hourly"
            worksheet.update_cell(first_unchecked_row, payment_type_col, payment_type)

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            prices = soup.find_all('strong', attrs={'data-v-8d6ae40e': True})
            
            if len(prices) == 2:
                min_price = prices[0].get_text().strip()
                max_price = prices[1].get_text().strip()
                hourly_rate = f"{min_price} - {max_price}"
                worksheet.update_cell(first_unchecked_row, price_col, hourly_rate)
            else:
                worksheet.update_cell(first_unchecked_row, price_col, "No hourly rate provided")
        except:
            try:
                budget_element = driver.find_element(By.CSS_SELECTOR, "[data-v-8d6ae40e]")
                payment_type = "Fixed Price"
                worksheet.update_cell(first_unchecked_row, payment_type_col, payment_type)

                price_element = budget_element.find_element(By.TAG_NAME, "strong")
                price = price_element.text
                worksheet.update_cell(first_unchecked_row, price_col, price)
            except:
                pass

        try:
            # Scrape skills
            all_skills = []
            skills_sections = driver.find_elements(By.CSS_SELECTOR, "div.skills-list.mt-3")
            for section in skills_sections:
                skill_elements = section.find_elements(By.CSS_SELECTOR, "span.air3-badge.air3-badge-highlight.badge.disabled")
                section_skills = [skill.text for skill in skill_elements]
                all_skills.extend(section_skills)

            unique_skills = list(set(all_skills))
            if unique_skills:
                skills_text = ", ".join(unique_skills).strip(", ")  # Remove extra commas
            else:
                skills_text = "No skills listed"
                
            worksheet.update_cell(first_unchecked_row, skills_col, skills_text)
        except:
            pass

        try:
            # Scrape Duration
            duration_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='duration2'] + strong")
            duration = duration_element.text
            worksheet.update_cell(first_unchecked_row, duration_col, duration)
        except:
            pass

        try:
            # Scrape Experience Level
            experience_level_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='expertise'] + strong")
            experience_level = experience_level_element.text
            worksheet.update_cell(first_unchecked_row, experience_level_col, experience_level)
        except:
            pass

        try:
            # Scrape Project Type (e.g., Ongoing project, etc.)
            project_type_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='briefcase-outlined'] + strong")
            project_type = project_type_element.text
            worksheet.update_cell(first_unchecked_row, project_type_col, project_type)
        except:
            pass

        # Scrape Activity Section
        try:
            activity_section = driver.find_element(By.CSS_SELECTOR, "ul.client-activity-items.list-unstyled.visitor")

            try:
                proposals = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Proposals')]/following-sibling::span[@class='value']").text
                worksheet.update_cell(first_unchecked_row, proposals_col, proposals)
            except:
                pass

            try:
                last_viewed = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Last viewed by client')]/following-sibling::span[@class='value']").text
                worksheet.update_cell(first_unchecked_row, last_viewed_col, last_viewed)
            except:
                pass

            try:
                interviewing = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Interviewing')]/following-sibling::div[@class='value']").text
                worksheet.update_cell(first_unchecked_row, interviewing_col, interviewing)
            except:
                pass

            try:
                invites_sent = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Invites sent')]/following-sibling::div[@class='value']").text
                worksheet.update_cell(first_unchecked_row, invites_sent_col, invites_sent)
            except:
                pass

            try:
                unanswered_invites = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Unanswered invites')]/following-sibling::div[@class='value']").text
                worksheet.update_cell(first_unchecked_row, unanswered_invites_col, unanswered_invites)
            except:
                pass
        except:
            pass

        # Scrape About Client Section
        try:
            about_client_section = driver.find_element(By.CSS_SELECTOR, "div.cfe-about-client-v2")

            try:
                member_since = about_client_section.find_element(By.CSS_SELECTOR, "div[data-qa='client-contract-date'] small").text.replace("Member since ", "")
                worksheet.update_cell(first_unchecked_row, member_since_col, member_since)
            except:
                pass

            try:
                location_city = about_client_section.find_element(By.CSS_SELECTOR, "li[data-qa='client-location'] strong").text
                location_time = about_client_section.find_element(By.CSS_SELECTOR, "li[data-qa='client-location'] div span:nth-child(1)").text
                location = f"{location_time}, {location_city}".strip(", ")
                worksheet.update_cell(first_unchecked_row, location_col, location)
            except:
                pass

            try:
                total_spent = about_client_section.find_element(By.CSS_SELECTOR, "strong[data-qa='client-spend'] span").text
                worksheet.update_cell(first_unchecked_row, total_spent_col, total_spent)
            except:
                pass

            try:
                hires = about_client_section.find_element(By.CSS_SELECTOR, "div[data-qa='client-hires']").text
                worksheet.update_cell(first_unchecked_row, hires_col, hires)
            except:
                pass

            try:
                client_type = about_client_section.find_element(By.CSS_SELECTOR, "li[data-qa='client-company-profile'] strong").text
                company_size = about_client_section.find_element(By.CSS_SELECTOR, "li[data-qa='client-company-profile'] div[data-qa='client-company-profile-size']").text
                client_type_details = f"{client_type}, {company_size}"
            except:
                try:
                    client_type_details = about_client_section.find_element(By.CSS_SELECTOR, "li[data-qa='client-company-profile']").text
                except:
                    client_type_details = "Individual client"
            
            worksheet.update_cell(first_unchecked_row, client_type_col, client_type_details)

        except:
            pass

        # Mark the "Done" column as TRUE for this row after completing scraping
        worksheet.update_cell(first_unchecked_row, done_col, "TRUE")

        # Close the driver after completing scraping for one URL
        driver.quit()

print("Scraping completed.")


Stopping at row 3, as it's already marked 'Done'
Scraping completed.


Optimizing Speed:

In [ ]:
import time
import gspread
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service


# Authenticate and access the Google Sheet
gc = gspread.service_account(filename=sheet_path)
sh = gc.open_by_url(sheet_url)
worksheet = sh.get_worksheet(0)

# Retrieve the column headers and map them to column indices
column_headers = worksheet.row_values(2)
columns = {header: idx + 1 for idx, header in enumerate(column_headers)}

# Assign column numbers to variables for easier access
name_col = columns.get("Name")
url_col = columns.get("URL")
title_col = columns.get("Title")
payment_type_col = columns.get("Payment Type")
price_col = columns.get("Price")
project_type_col = columns.get("Project Type")
skills_col = columns.get("Skills")
proposals_col = columns.get("Proposals")
last_viewed_col = columns.get("Last viewed")
interviewing_col = columns.get("Interviewing")
invites_sent_col = columns.get("Invites sent")
unanswered_invites_col = columns.get("Unanswered invites")
done_col = columns.get("Done")
experience_level_col = columns.get("Experience Level")
duration_col = columns.get("Duration")
member_since_col = columns.get("Member Since")
location_col = columns.get("Location")
total_spent_col = columns.get("Total Spent")
hires_col = columns.get("Hires")
client_type_col = columns.get("Client Type")
description_col = columns.get("Description")

# Get the URLs and Done status from the relevant columns
urls_column = worksheet.col_values(url_col)[2:]  # From row 3 onward
done_column = worksheet.col_values(done_col)[2:]  # From row 3 onward

# Loop through the URLs, skipping rows marked as "TRUE" in the Done column
for i, (url, done) in enumerate(zip(urls_column, done_column)):
    first_unchecked_row = i + 3  # Since data starts from row 3

    # If the URL is empty, stop the script (end of data)
    if not url:
        print(f"Stopping at row {first_unchecked_row} due to no URL.")
        break

    # Skip rows that are already marked as "TRUE" (done)
    if done.upper() == "TRUE":
        print(f"Skipping row {first_unchecked_row} as it's already marked 'Done'")
        continue

    # If URL is not empty, proceed to scrape
    print(f"Processing row {first_unchecked_row} - {url}")
    
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    
    driver.get(url)
    time.sleep(3)  # Give page some time to load

    try:
        # Scrape the job title
        job_title_element = driver.find_element(By.CSS_SELECTOR, "h4.m-0")
        job_title = job_title_element.text
        worksheet.update_cell(first_unchecked_row, title_col, job_title)
    except:
        pass

    try:
        # Scrape job description
        description_element = driver.find_element(By.CSS_SELECTOR, "div.break.mt-2 p.text-body-sm")
        job_description = description_element.text
        worksheet.update_cell(first_unchecked_row, description_col, job_description)
    except:
        pass

    try:
        # Check if the job is hourly or fixed price
        hourly_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='clock-hourly']")
        payment_type = "Hourly"
        worksheet.update_cell(first_unchecked_row, payment_type_col, payment_type)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        prices = soup.find_all('strong', attrs={'data-v-8d6ae40e': True})
        
        if len(prices) == 2:
            min_price = prices[0].get_text().strip()
            max_price = prices[1].get_text().strip()
            hourly_rate = f"{min_price} - {max_price}"
            worksheet.update_cell(first_unchecked_row, price_col, hourly_rate)
        else:
            worksheet.update_cell(first_unchecked_row, price_col, "No hourly rate provided")
    except:
        try:
            budget_element = driver.find_element(By.CSS_SELECTOR, "[data-v-8d6ae40e]")
            payment_type = "Fixed Price"
            worksheet.update_cell(first_unchecked_row, payment_type_col, payment_type)

            price_element = budget_element.find_element(By.TAG_NAME, "strong")
            price = price_element.text
            worksheet.update_cell(first_unchecked_row, price_col, price)
        except:
            pass

    try:
        # Scrape skills
        all_skills = []
        skills_sections = driver.find_elements(By.CSS_SELECTOR, "div.skills-list.mt-3")
        for section in skills_sections:
            skill_elements = section.find_elements(By.CSS_SELECTOR, "span.air3-badge.air3-badge-highlight.badge.disabled")
            section_skills = [skill.text for skill in skill_elements]
            all_skills.extend(section_skills)

        unique_skills = list(set(all_skills))
        if unique_skills:
            skills_text = ", ".join(unique_skills).strip(", ")  # Remove extra commas
        else:
            skills_text = "No skills listed"
            
        worksheet.update_cell(first_unchecked_row, skills_col, skills_text)
    except:
        pass

    try:
        # Scrape Duration
        duration_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='duration2'] + strong")
        duration = duration_element.text
        worksheet.update_cell(first_unchecked_row, duration_col, duration)
    except:
        pass

    try:
        # Scrape Experience Level
        experience_level_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='expertise'] + strong")
        experience_level = experience_level_element.text
        worksheet.update_cell(first_unchecked_row, experience_level_col, experience_level)
    except:
        pass

    try:
        # Scrape Project Type (e.g., Ongoing project, etc.)
        project_type_element = driver.find_element(By.CSS_SELECTOR, "div[data-cy='briefcase-outlined'] + strong")
        project_type = project_type_element.text
        worksheet.update_cell(first_unchecked_row, project_type_col, project_type)
    except:
        pass

    # Scrape Activity Section
    try:
        activity_section = driver.find_element(By.CSS_SELECTOR, "ul.client-activity-items.list-unstyled.visitor")

        try:
            proposals = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Proposals')]/following-sibling::span[@class='value']").text
            worksheet.update_cell(first_unchecked_row, proposals_col, proposals)
        except:
            pass

        try:
            last_viewed = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Last viewed by client')]/following-sibling::span[@class='value']").text
            worksheet.update_cell(first_unchecked_row, last_viewed_col, last_viewed)
        except:
            pass

        try:
            interviewing = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Interviewing')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, interviewing_col, interviewing)
        except:
            pass

        try:
            invites_sent = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Invites sent')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, invites_sent_col, invites_sent)
        except:
            pass

        try:
            unanswered_invites = activity_section.find_element(By.XPATH, ".//span[contains(text(), 'Unanswered invites')]/following-sibling::div[@class='value']").text
            worksheet.update_cell(first_unchecked_row, unanswered_invites_col, unanswered_invites)
        except:
            pass
    except:
        pass

    # Scrape About Client Section
    try:
        about_client_section = driver.find_element(By.CSS_SELECTOR, "div.cfe-about-client-v2")

        try:
            member_since = about_client_section.find_element(By.CSS_SELECTOR, "div[data-qa='client-contract-date'] small").text.replace("Member since ", "")
            worksheet.update_cell(first_unchecked_row, member_since_col, member_since)
        except:
            pass

        try:
            location_city = about_client_section.find_element(By.CSS_SELECTOR, "li[data-qa='client-location'] strong").text
            location_time = about_client_section.find_element(By.CSS_SELECTOR, "li[data-qa='client-location'] div span:nth-child(1)").text
            location = f"{location_time}, {location_city}".strip(", ")
            worksheet.update_cell(first_unchecked_row, location_col, location)
        except:
            pass

        try:
            total_spent = about_client_section.find_element(By.CSS_SELECTOR, "strong[data-qa='client-spend'] span").text
            worksheet.update_cell(first_unchecked_row, total_spent_col, total_spent)
        except:
            pass

        try:
            hires = about_client_section.find_element(By.CSS_SELECTOR, "div[data-qa='client-hires']").text
            worksheet.update_cell(first_unchecked_row, hires_col, hires)
        except:
            pass

        try:
            client_type = about_client_section.find_element(By.CSS_SELECTOR, "li[data-qa='client-company-profile'] strong").text
            company_size = about_client_section.find_element(By.CSS_SELECTOR, "li[data-qa='client-company-profile'] div[data-qa='client-company-profile-size']").text
            client_type_details = f"{client_type}, {company_size}"
        except:
            try:
                client_type_details = about_client_section.find_element(By.CSS_SELECTOR, "li[data-qa='client-company-profile']").text
            except:
                client_type_details = "Individual client"
        
        worksheet.update_cell(first_unchecked_row, client_type_col, client_type_details)

    except:
        pass

    # Mark the "Done" column as TRUE for this row after completing scraping
    worksheet.update_cell(first_unchecked_row, done_col, "TRUE")

    # Close the driver after completing scraping for one URL
    driver.quit()

    # Sleep for 3 seconds before moving to the next URL (to avoid CAPTCHA issues)
    time.sleep(3)

print("Scraping completed.")


Skipping row 3 as it's already marked 'Done'
Processing row 4 - https://www.upwork.com/nx/search/jobs/details/~01afac30d80f84e9f3?is_sts_vector_search_result=false&nav_dir=pop&payment_verified=1&proposals=0-4&sort=recency&subcategory2_uid=1044578476142100480,531770282593251334,531770282601639953&pageTitle=Job%20Details&_modalInfo=%5B%7B%22navType%22%3A%22slider%22,%22title%22%3A%22Job%20Details%22,%22modalId%22%3A%221722106662348%22%7D%5D
Processing row 5 - https://www.upwork.com/jobs/~01ec3a7396fa5b7942


KeyboardInterrupt: 